In [47]:
%matplotlib inline
%load_ext memory_profiler

import os
import time
import datetime
import sys
import glob
import pickle
import scanpy as sc
from scipy import sparse
import numpy as np
from sklearn.model_selection import train_test_split

sc.settings.verbosity=2

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


#### Load AnnData obj

In [48]:
pdfp = '/home/ngr4/project/scni/data/processed_200108/'
pfp = '/home/ngr4/project/scni/results/'

def loader(fname,fpath,backed=None) : 
    start = time.time()
    adata = sc.read_h5ad(filename=os.path.join(fpath,fname),backed=backed)
    print('loaded @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to load data'.format(time.time()-start))
    return adata

def writer(fname,fpath,AnnData) :
    start = time.time()
    AnnData.write(os.path.join(fpath,fname))
    print('saved @'+datetime.datetime.now().strftime('%y%m%d.%H:%M:%S'))
    print('took {:.2f}-s to save data'.format(time.time()-start))
    

if True :
    # load personal
    fname='adata_transduction.h5ad'
    %memit adata = loader(fname,pdfp)

loaded @200604.10:56:07
took 11.03-s to load data
peak memory: 31671.86 MiB, increment: 11224.64 MiB


# Induction

Take 40p of the data

In [49]:
# split the data AND kick out mock infected cells as these are noise
idx_train, idx_nottrain = train_test_split(adata.obs.index, train_size=0.4)
tdata = sc.AnnData(X=adata[(adata.obs.index.isin(idx_train)),:].X,
                          obs=adata[(adata.obs.index.isin(idx_train)),:].obs)
idx_val, idx_test = train_test_split(idx_nottrain, train_size=0.25)
idx_test, _ = train_test_split(idx_test, train_size=0.20)
val = sc.AnnData(X=adata[(adata.obs.index.isin(idx_val)),:].X, 
                  obs=adata[(adata.obs.index.isin(idx_val)),:].obs)
test = sc.AnnData(X=adata[(adata.obs.index.isin(idx_test)),:].X, 
                  obs=adata[(adata.obs.index.isin(idx_test)),:].obs)

def graph_pp(AnnData, bbknn=False):
    sc.tl.pca(AnnData, n_comps=100)
    if bbknn:
        sc.external.pp.bbknn(AnnData) # use default params
    else:
        sc.pp.neighbors(AnnData, n_pcs=100, n_neighbors=30)
    return AnnData

# make graph
tdata = graph_pp(tdata, bbknn=False)
val = graph_pp(val, bbknn=False)
test = graph_pp(test, bbknn=False)

if True:
    feature_names = adata.var_names.to_list()
    del adata

computing PCA
    with n_comps=100
    finished (0:05:08)
computing neighbors
    using 'X_pca' with n_pcs = 100


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/numba/typed_passes.py:271: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../conda_envs/rnavel/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  state.func_ir.loc))
/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../con

    finished (0:01:17)
computing PCA
    with n_comps=100
    finished (0:02:11)
computing neighbors
    using 'X_pca' with n_pcs = 100


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/numba/typed_passes.py:271: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../conda_envs/rnavel/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  state.func_ir.loc))


    finished (0:00:20)
computing PCA
    with n_comps=100
    finished (0:01:16)
computing neighbors
    using 'X_pca' with n_pcs = 100


/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/numba/typed_passes.py:271: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../conda_envs/rnavel/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  state.func_ir.loc))


    finished (0:00:11)


## Encoding 

Select tasks for prediction

1. yctype 
2. severe


In [50]:
# encode main label
# ref: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE145926
batch_encoder = {v:i for i,v in enumerate(tdata.obs['batch'].unique())}

tdata.obs['ybatch'] = tdata.obs['batch'].map(batch_encoder)
val.obs['ybatch'] = val.obs['batch'].map(batch_encoder)
test.obs['ybatch'] = test.obs['batch'].map(batch_encoder)

tdata.obs['yms'] = tdata.obs['MS'].astype(int)
val.obs['yms'] = val.obs['MS'].astype(int)
test.obs['yms'] = test.obs['MS'].astype(int)

# encode ctype
ctype_encoder = {v:i for i,v in enumerate(tdata.obs['louvain'].unique())}
tdata.obs['yctype'] = tdata.obs['louvain'].map(ctype_encoder)
val.obs['yctype'] = val.obs['louvain'].map(ctype_encoder)
test.obs['yctype'] = test.obs['louvain'].map(ctype_encoder)


In [51]:
# create dictionary
def dictthat(AnnData, feat_names=None, gene_ranger=True):
    """Prep dictionary for export.
    
    If gene_ranger, divide by zero can occur for 
    non-expressing genes. Thus, will floor those
    to 0.
    
    NOTE: customization re:y to predict is highly
    dependent on user input. ERGO, modify this 
    
    Arguments:
        AnnData (sc.AnnData): with graph stuff
        
    Returns:
        dict
    """
    if gene_ranger:
        # each gene in [0,1], divide by zeros to 0
        minimum = AnnData.X.min(axis=0)
        maximum = AnnData.X.max(axis=0)
        if isinstance(minimum, np.ndarray):
            num = AnnData.X - minimum
        else:
            num = AnnData.X - minimum.todense()
        if isinstance((maximum - minimum), np.ndarray):
            denom = (maximum - minimum)
        else:
            denom =  (maximum - minimum).todense()
        xhat = np.divide(num, denom, out=np.zeros_like(num), where=denom!=0) 
    else:
        # matrix in [0,1]
        xhat = (AnnData.X - AnnData.X.min()) / (AnnData.X.max() - AnnData.X.min())
        
    

    gdata = {'X':xhat,
             'adj':AnnData.uns['neighbors']['connectivities']+sparse.diags([1]*AnnData.shape[0], format='csr')}
    if feat_names is None:
        gdata['feature_names'] = AnnData.var_names.to_list()
    else:
        gdata['feature_names'] = feat_names
    gdata['cell_id'] = AnnData.obs.index.to_list()
    for col in AnnData.obs.columns:
        gdata[col] = AnnData.obs[col].to_list()
    
    return gdata

gdata_train = dictthat(tdata, feat_names=feature_names)
gdata_val = dictthat(val, feat_names=feature_names)
gdata_test  = dictthat(test, feat_names=feature_names)



/gpfs/ycga/project/dijk/ngr4/conda_envs/rnavel/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.


In [52]:
# export
def pklthat(gdata, fname, fpath=pdfp): 
    with open(os.path.join(fpath,fname),'wb') as f :
        pickle.dump(gdata, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.close()

pklthat(gdata_train, 'scni_train_knn200604.pkl')
pklthat(gdata_val, 'scni_val_knn200604.pkl')
pklthat(gdata_test, 'scni_test_knn200604.pkl')

# clean
if True:
    del tdata, test, gdata_train, gdata_test

In [53]:
pdfp

'/home/ngr4/project/scni/data/processed_200108/'